In [10]:
import numpy as np
import pandas as pd
from glob import glob
from os.path import join
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
import torch.optim as optim

from torchvision.transforms import RandomRotation, RandomHorizontalFlip, RandomResizedCrop,GaussianBlur
# yes

class AgeDataset(torch.utils.data.Dataset):
    def __init__(self, data_path, annot_path, train=True):
        super(AgeDataset, self).__init__()

        self.annot_path = annot_path
        self.data_path = data_path
        self.train = train

        self.ann = pd.read_csv(annot_path)
        if train:
            # Calculate the frequency of each age
            age_counts = self.ann['age'].value_counts()
            age_counts = self.ann['age'].value_counts().head(70)

            # Calculate the average number of samples per age
            threshold = int(age_counts.mean())

            # Find the ages that are overrepresented
            overrepresented_ages = age_counts[age_counts > threshold].index

            # Remove random samples from overrepresented ages
            for age in overrepresented_ages:
                over_samples = self.ann[(self.ann['age'] == age)]
                drop_indices = np.random.choice(over_samples.index, size=(len(over_samples) - threshold), replace=False)
                self.ann = self.ann.drop(drop_indices)

            # Reset the index after dropping rows
            self.ann.reset_index(drop=True, inplace=True)

        self.files = self.ann['file_id']
        if train:
            self.ages = self.ann['age']
        self.transform = self._transform(224)

    @staticmethod
    def _convert_image_to_rgb(image):
        return image.convert("RGB")


    def _transform(self, n_px):
        if self.train:
            # Data augmentation for training
            transform = Compose([
                RandomRotation(degrees=15),
                RandomResizedCrop(size=n_px,
                                  scale=(0.8, 1.0)),
                RandomHorizontalFlip(),
                Resize(n_px),
                GaussianBlur(kernel_size=3),
                self._convert_image_to_rgb,
                ToTensor(),
                Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
            ])
        else:
            # Validation/test transformation
            transform = Compose([
                Resize(n_px),
                self._convert_image_to_rgb,
                ToTensor(),
                Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
            ])
        return transform

    def read_img(self, file_name):
        im_path = join(self.data_path, file_name)
        img = Image.open(im_path)
        img = self.transform(img)
        return img

    def __getitem__(self, index):
        file_name = self.files[index]
        img = self.read_img(file_name)
        if self.train:
            age = self.ages[index]
            return img, age
        else:
            return img

    def __len__(self):
        return len(self.files)


train_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train'
train_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/train.csv'
train_dataset = AgeDataset(train_path, train_ann, train=True)


test_path = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/test'
test_ann = '/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv'
test_dataset = AgeDataset(test_path, test_ann, train=False)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
train_indices, val_indices = train_test_split(list(range(len(train_dataset))), test_size=0.2, random_state=42)
# Create training and validation samplers
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

# Create data loaders for training and validation sets
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, sampler=val_sampler)

In [11]:
print(len(train_dataset))

14290


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [16]:
def predict_test(loader, model):
    model.eval()
    predictions = []

    for img in tqdm(loader):
        img = img.to(device)
        pred = model(img)
        predictions.extend(pred.flatten().detach().tolist())

    return predictions


In [20]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch

# Load pre-trained ResNet-50 model
model = models.wide_resnet50_2( weights='Wide_ResNet50_2_Weights.DEFAULT')

# Unfreeze additional layers for fine-tuning
for name, param in model.named_parameters():
    if "layer3" in name or "layer4" in name or "fc" in name:  # Unfreeze layers
        param.requires_grad = True
    else:
        param.requires_grad = False

# Modify the last layer for age detection task
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.3),          # Add dropout layer
    nn.Linear(num_ftrs, 1024),  # Add additional fully connected layer
    nn.ReLU(),
    nn.Dropout(0.3),           # Add dropout layer
    nn.Linear(1024, 1)          # Output 1 value for age prediction
)

# Define loss function and optimizer
criterion = nn.MSELoss()

optimizer = optim.NAdam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

# Use a different learning rate scheduler
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Decrease learning rate every 5 epochs

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), strid

In [21]:
# Early stopping details
n_epochs_stop = 3
min_val_loss = float('inf')
epochs_no_improve = 0

# Training loop
num_epochs = 12 # Increase epochs to 20
best_model_path='model'
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs.squeeze(), labels.float()).item() * inputs.size(0)

    val_loss /= len(val_loader.dataset)
    print(f"Validation Loss: {val_loss:.4f}")

    # Step the scheduler
    scheduler.step()

    # Early stopping
    if val_loss < min_val_loss:
        epochs_no_improve = 0
        min_val_loss = val_loss
        torch.save(model,best_model_path)
    else:
        epochs_no_improve += 1
        # Check early stopping condition
        if epochs_no_improve == n_epochs_stop:
            print('Early stopping!')
            break
    # Generate predictions for the test set
    test_predictions = predict_test(test_loader, model)
    print(test_predictions[:50])



Epoch [1/12], Training Loss: 140.8833
Validation Loss: 21.3405


100%|██████████| 31/31 [00:10<00:00,  2.85it/s]


[59.509735107421875, 37.38518142700195, 24.847087860107422, 44.62004470825195, 29.76712417602539, 32.49863052368164, 90.79975891113281, 37.04011917114258, 42.491153717041016, 54.87962341308594, 42.74684524536133, 60.79868698120117, 52.77585983276367, 26.981334686279297, 40.59995651245117, 50.87338638305664, 10.152323722839355, 78.1843490600586, 69.8510513305664, 74.55427551269531, 34.0316047668457, 65.31210327148438, 32.12525177001953, 32.71180725097656, 31.89166831970215, 26.307798385620117, 30.471603393554688, 4.339181900024414, 2.2408552169799805, 23.858598709106445, 29.13344955444336, 47.27702713012695, 8.069622039794922, 1.4184620380401611, 14.954499244689941, 26.416765213012695, 26.81416130065918, 56.39934158325195, 7.577702045440674, 55.13795852661133, 26.13857078552246, 25.771474838256836, 28.26849937438965, 55.185611724853516, 14.229392051696777, 37.09695053100586, 21.43033218383789, 24.689777374267578, 15.998499870300293, 35.61384963989258]
Epoch [2/12], Training Loss: 77.658

100%|██████████| 31/31 [00:10<00:00,  2.90it/s]


[56.64539337158203, 43.07380676269531, 33.493507385253906, 67.68994140625, 26.964885711669922, 26.93781089782715, 85.41915893554688, 37.46620559692383, 45.2634162902832, 66.0262451171875, 29.001951217651367, 62.891056060791016, 58.262550354003906, 31.886260986328125, 59.77154541015625, 50.829307556152344, 15.02436351776123, 71.48870849609375, 59.504337310791016, 51.02264404296875, 29.7564754486084, 68.7247085571289, 34.62931823730469, 25.152210235595703, 40.027984619140625, 27.37067413330078, 41.873531341552734, 13.994391441345215, 5.828170299530029, 21.974166870117188, 37.00944137573242, 58.16190719604492, 10.703189849853516, 3.5569660663604736, 31.046606063842773, 24.773595809936523, 27.275556564331055, 54.655277252197266, 6.593519687652588, 54.11799240112305, 27.935495376586914, 25.172462463378906, 41.539310455322266, 55.57245635986328, 21.807422637939453, 52.384735107421875, 25.313074111938477, 35.71965789794922, 19.26053237915039, 40.12969970703125]
Epoch [3/12], Training Loss: 65

...

[59.73887634277344, 44.68575668334961, 34.55857467651367, 56.355037689208984, 28.596269607543945, 24.798152923583984, 79.60721588134766, 39.884376525878906, 50.14284133911133, 64.31661224365234, 42.87904357910156, 58.73067855834961, 68.39476013183594, 32.65621566772461, 56.79738998413086, 53.02525329589844, 11.507329940795898, 81.12517547607422, 64.9924545288086, 55.76268768310547, 29.907981872558594, 63.693756103515625, 34.126625061035156, 29.588481903076172, 41.48200607299805, 28.768722534179688, 37.04925537109375, 11.587740898132324, 6.04197359085083, 25.224620819091797, 32.93180465698242, 52.71287536621094, 10.79174518585205, 4.776249885559082, 30.707260131835938, 25.945297241210938, 28.21657371520996, 61.62000274658203, 3.5640335083007812, 47.45073318481445, 30.900381088256836, 26.04640769958496, 36.9650764465332, 53.90382385253906, 22.03009605407715, 45.81255340576172, 23.76005744934082, 26.479509353637695, 18.71438217163086, 45.48811721801758]
Epoch [11/12], Training Loss: 27.48

100%|██████████| 31/31 [00:10<00:00,  2.87it/s]


[59.24359130859375, 44.1624870300293, 34.98701477050781, 58.598899841308594, 28.432912826538086, 24.638031005859375, 80.14878845214844, 40.433780670166016, 50.0328254699707, 64.09835052490234, 43.89369583129883, 58.65237045288086, 68.73860168457031, 32.14096450805664, 57.46564865112305, 52.9005241394043, 10.209321975708008, 80.72593688964844, 65.29977416992188, 56.489986419677734, 29.740495681762695, 64.02643585205078, 34.90105056762695, 29.547815322875977, 40.798954010009766, 28.65423011779785, 37.48273468017578, 10.351123809814453, 5.194167137145996, 24.436735153198242, 32.351375579833984, 53.83036422729492, 9.886926651000977, 4.350317001342773, 31.398136138916016, 25.44474220275879, 27.730093002319336, 62.89369583129883, 3.3850440979003906, 46.88565444946289, 31.580810546875, 26.121227264404297, 36.51587677001953, 52.5238151550293, 21.91505241394043, 46.99072265625, 23.77555274963379, 26.338531494140625, 18.861268997192383, 45.19093322753906]
Epoch [12/12], Training Loss: 27.2002
Va

100%|██████████| 31/31 [00:10<00:00,  2.86it/s]

[58.098602294921875, 43.3123664855957, 34.33319091796875, 58.11607360839844, 28.22696876525879, 24.6635684967041, 79.91337585449219, 40.25537872314453, 49.30252456665039, 63.54266357421875, 43.11788558959961, 57.92616271972656, 68.75751495361328, 31.889883041381836, 57.140892028808594, 52.14873504638672, 10.425718307495117, 80.2174301147461, 64.89244842529297, 55.80400085449219, 29.496437072753906, 62.52437210083008, 34.30522155761719, 29.304893493652344, 40.225379943847656, 28.31818962097168, 37.027957916259766, 10.706559181213379, 5.283134937286377, 24.218141555786133, 31.78911781311035, 53.0847053527832, 10.432816505432129, 4.6051788330078125, 31.59854507446289, 25.170507431030273, 27.47118377685547, 62.540138244628906, 3.3861536979675293, 45.89964294433594, 31.384140014648438, 25.997766494750977, 36.00605773925781, 51.30144119262695, 21.907331466674805, 47.31787872314453, 23.800939559936523, 25.159412384033203, 18.841751098632812, 44.29779052734375]


In [22]:
def predict_test(loader, model):
    model.eval()
    predictions = []

    for img in tqdm(loader):
        img = img.to(device)
        pred = model(img)
        predictions.extend(pred.flatten().detach().tolist())

    return predictions


preds = predict_test(test_loader, model)

submit = pd.read_csv('/kaggle/input/smai-24-age-prediction/content/faces_dataset/submission.csv')
submit['age'] = preds
submit.head()

submit.to_csv('baseline.csv',index=False)


100%|██████████| 31/31 [00:10<00:00,  2.83it/s]


In [23]:
print(preds)

[58.098602294921875, 43.3123664855957, 34.33319091796875, 58.11607360839844, 28.22696876525879, 24.6635684967041, 79.91337585449219, 40.25537872314453, 49.30252456665039, 63.54266357421875, 43.11788558959961, 57.92616271972656, 68.75751495361328, 31.889883041381836, 57.140892028808594, 52.14873504638672, 10.425718307495117, 80.2174301147461, 64.89244842529297, 55.80400085449219, 29.496437072753906, 62.52437210083008, 34.30522155761719, 29.304893493652344, 40.225379943847656, 28.31818962097168, 37.027957916259766, 10.706559181213379, 5.283134937286377, 24.218141555786133, 31.78911781311035, 53.0847053527832, 10.432816505432129, 4.6051788330078125, 31.59854507446289, 25.170507431030273, 27.47118377685547, 62.540138244628906, 3.3861536979675293, 45.89964294433594, 31.384140014648438, 25.997766494750977, 36.00605773925781, 51.30144119262695, 21.907331466674805, 47.31787872314453, 23.800939559936523, 25.159412384033203, 18.841751098632812, 44.29779052734375, 33.68459701538086, 53.8164100646